In [8]:
import pandas as pd
import tweepy
from datetime import datetime
from pprint import pprint

from config import consumer_key, consumer_secret, access_token, access_token_secret

In [9]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [35]:
# Target User
target_user = "@realDonaldTrump"



# Tweet Texts
tweet_texts = []
created_date =[]
retweets=[]
likes=[]


# Create a loop to iteratively run API requests
for x in range(1,5):

    # Get all tweets from home feed (for each page specified)
    public_tweets = api.user_timeline(target_user, page=x)

    # Loop through all tweets
    for tweet in public_tweets:

        # Print Tweet
        tweet_texts.append(tweet["text"])
        created_date.append(tweet["created_at"])
        retweets.append(tweet["retweet_count"])
        likes.append(tweet["favorite_count"])
        
trump_tweets = {
                "Tweet" : tweet_texts,
                "Created" : created_date,
                "Retweets" : retweets,
                "Likes" : likes
                }


# print(len(likes))

trump_tweets = pd.DataFrame(trump_tweets)
trump_tweets.head(5)




,Created,Likes,Retweets,Tweet
0,Sat Jun 23 20:25:06 +0000 2018,7358,1923,Thank you @NVGOP! #MAGA https://t.co/OGTR2j6PGG
1,Sat Jun 23 17:12:03 +0000 2018,40062,8611,Happy Birthday to Supreme Court Justice Claren...
2,Sat Jun 23 17:05:33 +0000 2018,67179,18208,It’s very sad that Nancy Pelosi and her sideki...
3,Sat Jun 23 17:03:03 +0000 2018,0,5724,RT @HuckabeeOnTBN: TONIGHT: the President of t...
4,Sat Jun 23 16:57:48 +0000 2018,36228,8173,Heading to Nevada to talk trade and immigratio...


In [36]:
keyword = ['earnings', 'election', 'futures', 'investor', 'low', 'market', 'nasdaq', 'north korea', 'stock',
'market', 'correction', 'trading', 'unemployment','wall street','china',
'bull', 'kim', 'price', 'high', 'corporate', 'global warming', 'tariff', 'tax cut', 'jobs', 'economy', 'trade', 'missile', 'nuclear', 'trade', 'jobs', 'wages',
'maga']

# location_data["Date"] = dt.now().strftime("%m/%d/%Y")
# Date = dt.now().strftime("%m/%d/%Y")

In [ ]:
 for tweet in trump_tweets:
        

        # Print Tweet
        tweet_texts.append(tweet["text"])
        created_date.append(tweet["created_at"])
        retweets.append(tweet["retweet_count"])
        likes.append(tweet["favorite_count"])